In [1]:
import yaml
import os
from langchain_anthropic import ChatAnthropic
from sentence_transformers import SentenceTransformer
from matching_functions import find_closest_documents

model = SentenceTransformer("nomic-ai/nomic-embed-text-v1", trust_remote_code=True)

with open('config.yml', 'r') as file:
    config = yaml.safe_load(file)
anthropic_key = config["anthropic_key"]
os.environ["ANTHROPIC_API_KEY"] = anthropic_key
chat = ChatAnthropic(temperature=0, model_name="claude-3-haiku-20240307")

<All keys matched successfully>


In [2]:
from wikipedia_functions import get_random_page, get_page_links, get_page_content, check_wikipedia_pages_existence

start_page, end_page = get_random_page(2)
end_page_full_content = get_page_content(end_page)
print(start_page)
print(end_page)

Mainwaringia leithii
Survivor: South Pacific


In [3]:
start_links = get_page_links(start_page)
checked_links = check_wikipedia_pages_existence(start_links)
valid_links = [link for link in start_links if checked_links[link]]

In [4]:
from chain_functions import get_crawler_chain, get_summarize_chain, get_explain_links_chain, get_broad_links_chain

crawler_chain = get_crawler_chain(chat)
summarize_chain = get_summarize_chain(chat)
explain_links_chain = get_explain_links_chain(chat)
broad_links_chain = get_broad_links_chain(chat)

In [5]:
from answer_functions import answer_broad_links, answer_crawler, answer_explain_links, answer_summarize

end_page_content = answer_summarize(summarize_chain, end_page_full_content)
print(end_page_content)

Survivor: South Pacific was the 23rd season of the American reality TV series Survivor, filmed in Samoa and featuring 16 new contestants and two returning players, Ozzy Lusth and Benjamin "Coach" Wade. The season was won by Sophie Clarke, who defeated Coach and Albert Destrade in a 6-3-0 vote. Notable contestants included Brandon Hantz, Whitney Duncan, and Elyse Umemoto. Several contestants from this season, including Cochran, Dawn, and Ozzy, went on to appear in future Survivor seasons.


In [6]:
current_page = start_page
current_links = valid_links
forbidden_links = [start_page.strip()]
for i in range(10):
    closest_links = find_closest_documents(model, end_page, end_page_content, current_links)
    broad_links = answer_broad_links(broad_links_chain, current_links, end_page, end_page_content)
    current_links = list(set([link for link in closest_links + broad_links if link in current_links]))
    reasonings = answer_explain_links(explain_links_chain, current_links, end_page, end_page_content)
    current_page = answer_crawler(crawler_chain, current_page, current_links, reasonings, end_page, end_page_content).strip()
    if current_page != end_page:
        forbidden_links.append(current_page)
        found_links = get_page_links(current_page)
        checked_links = check_wikipedia_pages_existence(found_links)
        current_links = [link for link in found_links if checked_links[link] and link not in forbidden_links]
    else:
        print(f"Page reached in {i} iterations!")
        break

<reasoning>
Based on the provided information, the most relevant link to click on to get closer to the "Survivor: South Pacific" page appears to be "Animal". The Survivor TV show involves human contestants, and understanding the general characteristics and classification of animals could provide a useful foundation for understanding the context and dynamics of the show. While the "Gastropoda" and "Mollusca" links are also relevant to the given starting page, they are more specific and may not directly connect to the broader topic of the Survivor TV show. Therefore, the "Animal" link seems to be the most promising choice to make progress towards the target page.
</reasoning>

<output>Animal</output>
<reasoning>
Based on the provided information, the link that seems most relevant and useful to reach the goal page "Survivor: South Pacific" is "Taxonomy (biology)". Understanding the principles of biological taxonomy and the classification of living organisms can provide a broader context a